## Google Drive Authentication

In [2]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']


creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('drive', 'v3', credentials=creds)
    service_sheets = build('sheets', 'v4', credentials=creds)
    
    print("** Authentication Complete! **")

except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f'An error occurred: {error}')

** Authentication Complete! **


## Read Spreadsheets

In [5]:
from pathlib import Path
import pandas as pd

# Run this on update of either 1. layering order and 2. exclusions sheet
sheet = service_sheets.spreadsheets()

BABIES_TO_IMAGES_SPREADSHEET_ID = '1AgRmweMAzKK7MdHsT9i5ZaRyr7fdhMp2h3zmpab2A6k'

result = sheet.values().get(spreadsheetId=BABIES_TO_IMAGES_SPREADSHEET_ID, range='A:B').execute()
values = result.get('values', [])

df = pd.DataFrame(values, columns=["filename", "picked_images"])

In [22]:
import os
from PIL import Image, ImageDraw, ImageChops

transparent_babies_filepath = 'Transparent Babies'
if not os.path.isdir(transparent_babies_filepath):
    os.mkdir('Transparent Babies')
    
def generate_baby_image(picked_trait_images):
    
    # Layer the images 
    x, y = Image.open(picked_trait_images[0]).size
#     final_baby_image = Image.new('RGB', (x, y), (228, 150, 150))
    final_baby_image = Image.new('RGBA', (x, y))
    
    for trait_image in picked_trait_images:
        chosen_image = Image.open(trait_image)
        chosen_image = ImageChops.offset(chosen_image, 90, 0)
        final_baby_image.paste(chosen_image, (0, 0), chosen_image)
    
    # Crop to increase baby appearance
    final_baby_image = final_baby_image.crop((0, 140, final_baby_image.width - 140, final_baby_image.height))
    
    return final_baby_image

    
# get_transparent_baby_from_filename("0_lonely_baby.PNG")
def get_transparent_baby_from_filename(filename):
    final_baby_image_file = 'transparent_{:s}'.format(filename)
    if os.path.exists(os.path.join(transparent_babies_filepath, final_baby_image_file)):
        return
    
    picked_images = df.loc[df['filename'] == filename, 'picked_images'].item()
    picked_images_list = picked_images.split(',') 
    
    final_baby_image = generate_baby_image(picked_images_list[1:])
    final_baby_image.save(os.path.join(transparent_babies_filepath, final_baby_image_file))
    
    print("Completed " + final_baby_image_file)

## Run this to generate transparent babies!

In [23]:
get_transparent_baby_from_filename("1_lonely_baby.PNG")

Completed transparent_1_lonely_baby.PNG
